In [5]:
import cloudknot as ck
ck.set_region('us-west-2')

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [1]:
def afq_cam_can(subject):
    import AFQ.data as afqd
    import AFQ.api as api
    import AFQ.mask as afm
    import os.path as op
    import s3fs
    import packaging
    import cython

    local_bids_folder = "cam_can"
    bucket = "cam-can-mri"
    s3_prefix = ""
    s3_prefix_qsiprep = "derivatives/qsiprep"
    scope = 'qsiprep'

    # select subjects from qsiprep
    study_qsiprep = afqd.S3BIDSStudy(
        "Cam-CAN-qsiprep",
        bucket,
        s3_prefix_qsiprep,
        subjects=[subject],
        anon=False,
        random_seed=42)
    sub_name = study_qsiprep.subjects[0].subject_id

    study = afqd.S3BIDSStudy(
        "Cam-CAN",
        bucket,
        s3_prefix,
        subjects=sub_name,
        anon=False,
        random_seed=42)
    study.download(
        local_bids_folder,
        include_derivs=scope)
    
    tracking_params = {
        "n_seeds": 1, # also try 2
        "directions": "prob",
        "odf_model": "CSD", # also try dki
        "seed_mask": afm.MaskFile(
            "probseg",
            {"label": "WM",
            "space": None}), 
        "seed_threshold": 0.1,
        "stop_mask": afm.ScalarMask("dki_fa"),
        "stop_threshold": 0.2,
    }
    myafq = api.AFQ(
        local_bids_folder,
        dmriprep='all',
        brain_mask=afm.MaskFile(
            "mask",
            {"desc": "brain",
            "space": None}),
        scalars=['dki_md', 'dki_fa'],
        tracking_params=tracking_params,
        virtual_frame_buffer=True)
    myafq.export_all()
    
    myafq.upload_to_s3(s3fs.S3FileSystem(), op.join(bucket, s3_prefix, "derivatives/afq"))
    

In [11]:
#afq_cam_can(1)

In [6]:
afq_knot = ck.Knot(name='afq_cam_can-64gb-20826-5',
                  func=afq_cam_can,
                  base_image='libglxvfb:1',
                  image_github_installs="https://github.com/36000/pyAFQ.git@avoid_dipy_load_tractogram_bug#egg=pyAFQ[fury]",
                  pars_policies=('AmazonS3FullAccess',),
                  bid_percentage=100,
                  memory=64000)

In [7]:
inputs = [1, 2]

In [8]:
ft = afq_knot.map(inputs)

In [10]:
ck.set_region('us-west-2')
afq_knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
57214a1d-8997-4920-9f60-86251136e14e        afq-cam-can-64gb-20826-5-0        SUCCEEDED
